In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
internet_ds = pd.read_csv("../input/cicdarknet2020-internet-traffic/Darknet.CSV", keep_default_na = False,low_memory = False, error_bad_lines = False)
internet_ds

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
reduce_mem_usage(internet_ds)

In [ ]:
internet_ds.info()

In [ ]:
internet_ds.rename(columns = {"Label" : "Type", "Label.1" : "Subtype"}, inplace = True)
internet_ds

In [ ]:
internet_ds["Type"].unique()

In [ ]:
internet_ds["Subtype"].unique()

In [ ]:
internet_ds.groupby("Type")["Subtype"].unique()

In [ ]:
internet_ds.groupby("Type")["Subtype"].nunique()

In [ ]:
internet_ds["Subtype"].loc[internet_ds["Subtype"] == "AUDIO-STREAMING"] = "Audio-Streaming"
internet_ds["Subtype"].loc[internet_ds["Subtype"] == "File-transfer"] = "File-Transfer"
internet_ds["Subtype"].loc[internet_ds["Subtype"] == "Video-streaming"] = "Video-Streaming"

In [ ]:
internet_ds.groupby("Type")["Subtype"].unique()

In [ ]:
internet_ds.groupby("Type")["Subtype"].nunique()

In [ ]:
internet_ds.loc[:, "Encryption"] = ''
internet_ds

In [ ]:
internet_ds["Encryption"].loc[internet_ds["Type"] == "Non-Tor"] = "Standard"
internet_ds["Encryption"].loc[internet_ds["Type"] == "NonVPN"] = "Standard"

internet_ds["Encryption"].loc[internet_ds["Type"] == "Tor"] = "High"
internet_ds["Encryption"].loc[internet_ds["Type"] == "VPN"] = "High"

In [ ]:
internet_ds

In [ ]:
internet_ds.isnull().sum()


In [ ]:
internet_ds.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
#internet_ds.dropna(inplace=True)
internet_ds.fillna(0, inplace = True)

In [ ]:
internet_ds.isnull().sum()


In [ ]:
internet_ds_test = internet_ds.copy()
internet_ds_test

In [ ]:
#internet_ds_test.drop(["Type", "Subtype"],axis = 1,inplace = True)
internet_ds_test

In [ ]:
internet_ds_test.drop(["Timestamp"],axis = 1,inplace = True)
internet_ds_test

In [ ]:
#internet_ds_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

internet_ds_test = internet_ds_test.loc[:, internet_ds_test.apply(pd.Series.nunique) != 1]
internet_ds_test

In [ ]:
internet_ds_test.isnull().sum()


In [ ]:
internet_ds_test.dtypes


In [ ]:
internet_ds_test.drop(["Flow ID"],axis = 1,inplace = True)
internet_ds_test.drop(["Src IP"],axis = 1,inplace = True)
internet_ds_test.drop(["Dst IP"],axis = 1,inplace = True)
internet_ds_test.drop(["Src Port"],axis = 1,inplace = True)
internet_ds_test.drop(["Dst Port"],axis = 1,inplace = True)
internet_ds_test.drop(["Flow Duration"],axis = 1,inplace = True)

internet_ds_test.drop(["Idle Mean"],axis = 1,inplace = True)
internet_ds_test.drop(["Idle Std"],axis = 1,inplace = True)
internet_ds_test.drop(["Idle Max"],axis = 1,inplace = True)
internet_ds_test.drop(["Idle Min"],axis = 1,inplace = True)
internet_ds_test

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer as labBin

#label = labBin()
#internet_ds_encoded = pd.DataFrame(label.fit_transform(internet_ds_test[["Type","Subtype"]]), columns = label.classes_)
#internet_ds_encoded
internet_ds_test["Encryption"] = internet_ds_test["Encryption"].map({"Standard":0,"High":+1})
internet_ds_test

In [ ]:
from sklearn.model_selection import train_test_split as t_t_s
from sklearn.preprocessing import StandardScaler as stdScal



#X = internet_ds_test[["Type","Subtype"]]
X = internet_ds_test["Subtype"]

Y = internet_ds_test["Encryption"]
#X.drop(["Encryption"],axis = 1,inplace = True)


#X_std_scal = stdScal().fit_transform(X)

X_train, X_test, Y_train, Y_test = t_t_s(X, Y, test_size=0.30, random_state=42)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(sublinear_tf=True)

X_train_cnt = count_vect.fit_transform(X_train)
X_train_TF = transformer.fit_transform(X_train_cnt)
print(X_train_cnt.shape)
print(X_train_TF.shape)

X_test_cnt = count_vect.transform(X_test)
X_test_TF = transformer.transform(X_test_cnt)
print(X_test_cnt.shape)
print(X_test_TF.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFClass
ranForModel = RFClass(n_estimators=10, criterion = "entropy")
ranForModel.fit(X_train_TF, Y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report as class_re
from sklearn.metrics import confusion_matrix as c_m
ranForPredict = ranForModel.predict(X_test_TF)
print("Predicted Class:",ranForPredict)
print("Confusion Matrix:\n",c_m(Y_test ,ranForPredict))
print("Accuracy:", accuracy_score(Y_test ,ranForPredict))
print("F_score:", f1_score(Y_test ,ranForPredict))
print("Classification Report:\n",class_re(Y_test ,ranForPredict))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import classification_report as class_re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as t_t_s
from sklearn.preprocessing import OneHotEncoder as ohe
subtypes = ['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer', 'P2P', 'VOIP', 'Video-Streaming']
subtypes_ds = pd.DataFrame(subtypes, columns=['subtypes'])

encoder = ohe()


internet_ds_test["Subtype"] = internet_ds_test["Subtype"].astype('category')
internet_ds_test["Subtype_Encoded"] = internet_ds_test["Subtype"].cat.codes
#ds_encoded = pd.DataFrame(enc.fit_transform().toarry())


internet_ds_test


In [ ]:
ds_encoded = pd.DataFrame(encoder.fit_transform(internet_ds_test[['Subtype_Encoded']]).toarray())
ds_encoded

In [ ]:
#ds_encoded = pd.DataFrame(encoder.fit_transform(internet_ds_test[['Subtype_Encoded']]).toarray())

internet_ds_encoded = internet_ds_test[["Subtype","Subtype_Encoded","Type","Encryption"]].join(ds_encoded)

internet_ds_encoded

In [ ]:
#internet_ds_encoded.dropna(inplace=True)
#internet_ds_encoded.fillna(0, inplace=True)

internet_ds_encoded


In [ ]:
internet_ds_encoded.info()

In [ ]:
internet_ds_encoded.describe()

In [ ]:
internet_ds_encoded.isnull()

In [ ]:
subtypes = ['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer', 'P2P', 'VOIP', 'Video-Streaming']

internet_ds_encoded.rename(columns = {0 : "Audio-Streaming", 1 : "Browsing", 2 : "Chat", 3 : "Email", 4 : "File-Transfer", 5 : "P2P", 6 : "VOIP", 7 : "Video-Streaming"}, inplace = True)
internet_ds_encoded.info()

In [ ]:
subtype = ["Audio-Streaming", "Browsing", "Chat", "Email", "File-Transfer", "P2P", "VOIP", "Video-Streaming"]

X = internet_ds_encoded[subtype]
#X.drop()
X_std_scal = StandardScaler().fit_transform(X)
#Y = internet_ds_encoded["Type"]
Y = internet_ds_encoded["Encryption"]

X_train, X_test, Y_train, Y_test = t_t_s(X_std_scal, Y, test_size = 0.33, random_state = 42)

#DTreeClass = DecisionTreeClassifier(criterion = "entropy", random_state = 42, max_depth = 7)
DTreeClass = DecisionTreeClassifier(criterion = "entropy", random_state = 42)
#DTreeClass = DecisionTreeClassifier(random_state = 42)

DTreeClass.fit(X_train, Y_train)
Y_pred = DTreeClass.predict(X_test)
Y_scored = DTreeClass.score(X,Y)
Y_scored2 = DTreeClass.score(X_train,Y_train)
Y_scored3 = DTreeClass.score(X_test,Y_test)

In [ ]:
Y.unique()

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
#internet_ds_test["Encryption"] = internet_ds_test["Encryption"].map({"Standard":0,"High":+1})

#treePic = export_graphviz(DTreeClass, out_file = None, filled = True, rounded = True,special_characters = True, feature_names = subtypes ,class_names = ['Non-Tor','NonVPN'])
#treePic = export_graphviz(DTreeClass, out_file = None, filled = True, rounded = True,special_characters = True, feature_names = subtypes)
treePic = export_graphviz(DTreeClass, out_file = None, filled = True, rounded = True,special_characters = True, feature_names = subtypes ,class_names = ['Standard','High'])
#treePic = export_graphviz(DTreeClass, out_file = None, filled = True, rounded = True,special_characters = True, feature_names = subtypes ,class_names = ['Standard','High'])
#treePic = export_graphviz(DTreeClass, out_file = None, filled = True, rounded = True,special_characters = True, feature_names = subtypes ,class_names = ['High','Standard'])

graph = graphviz.Source(treePic)
graph.render("decision_tree", view = True)
graph

In [ ]:
#from sklearn.tree import DecisionTreeClassifier as DTreeClass
#dTree_entropy = DTreeClass(criterion = "entropy", random_state = 42)
#dTree_entropy.fit(X_train, Y_train)

#dTree_gigi = DTreeClass(criterion = "gigi", random_state = 42)
#dTree_gigi.fit(X_train, Y_train)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (18,6)
#plt.rcParams['figure.figsize'] = (20,28)

sns.histplot(data=internet_ds_encoded[internet_ds_encoded['Encryption'] == +1], x="Subtype", stat="probability")
plt.xlabel('')
plt.show()

In [ ]:
sns.histplot(data=internet_ds_encoded[internet_ds_encoded['Encryption'] == 0], x="Subtype", stat="probability")
plt.xlabel('')
plt.show()